In [ ]:
import random
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.linalg as la
import matplotlib.pyplot as plt

from tqdm import tqdm
from IPython.display import clear_output
from scipy.stats import norm,uniform,multivariate_normal
np.random.seed(10)

# A1. Implement at least one standard distribution (e.g. Dirichlet, Binomial, Gaussian etc.) (PMF/PDF) from scratch (using the formula, without libraries). Create at least 4 interesting plots for the PMF/PDF and 4 plots for the likelihood function. Analyze and reflect what they mean.

Gaussian distribution is the one that makes minimal assumption about a given data with a specific mean and standard deviation. Its formula comes from maximizing the entropy of a constraint distribution:

$$p(x)= \frac{1}{\sqrt{(2\pi)^{N}|\Sigma|}}e^{-\frac{(x-\mu)^{T}\Sigma^{-1}(x-\mu)}{2}}$$


In [ ]:
def gaussianDistribution(mean,sigma,boundary):
    precision=lambda sigma: np.linalg.inv(sigma)
    gaussian=lambda x1,x2: np.exp(-([x1, x2]-mean).T\
                                    @ precision(sigma)@([x1, x2]-mean))\
                                    /2/np.pi/np.sqrt(np.linalg.det(sigma))

    dim=50
    x = np.linspace(-boundary, boundary, dim)
    y = np.linspace(-boundary, boundary, dim)
    Z=np.zeros((dim,dim))
    for i,xi in enumerate(x):
        for j,yi in enumerate(y):
            Z[i,j] = gaussian(xi, yi)
    X, Y = np.meshgrid(x, y)
    return X, Y, Z

boundary=2
mean=np.array([0,0])
sigma = np.array([[1,0],[0,1]])
X, Y, Z=gaussianDistribution(mean,sigma,boundary)
plt.figure()
plt.contour(X, Y, Z.T, colors='black');
plt.title('Standard Multivariate Gaussian')
plt.xlabel('1-st dim')
plt.ylabel('2-nd dim')
plt.show()

boundary=3
mean=np.array([0,0])
sigma = np.array([[4,0],[0,2]])
X, Y, Z=gaussianDistribution(mean,sigma,boundary)
plt.figure()
plt.contour(X, Y, Z.T, colors='black');
plt.title('Non-Standard Multivariate Gaussian\n Diagonal covariance matrix')
plt.xlabel('1-st dim')
plt.ylabel('2-nd dim')
plt.show()


boundary=3
mean=np.array([0,0])
sigma = np.array([[4,2],[2,2]])
X, Y, Z=gaussianDistribution(mean,sigma,boundary)
plt.figure()
plt.contour(X, Y, Z.T, colors='black');
plt.title('Non-Standard Multivariate Gaussian\n  Positively correlated random-values')
plt.xlabel('1-st dim')
plt.ylabel('2-nd dim')
plt.show()


boundary=3
mean=np.array([0,0])
sigma = np.array([[4,-1],[-1,2]])
X, Y, Z=gaussianDistribution(mean,sigma,boundary)
plt.figure()
plt.contour(X, Y, Z.T, colors='black');
plt.title('Non-Standard Multivariate Gaussian\n  Negatively correlated random-values')
plt.xlabel('1-st dim')
plt.ylabel('2-nd dim')
plt.show()

###### The likelihood function for Gaussian 
Instead of working directly with the likelihood, we try to work with the log-likelihood for better numerical stability and easy computations.
The log-likelihood for both mean and covariance matrix is:

$$L(\mu,\Sigma)=-\frac{(x-\mu)^{T}\Sigma^{-1}(x-\mu)}{2}-\frac{1}{2}\{N*ln(2\pi)+ln(|\Sigma|)\}$$

In [ ]:
def meanLikelihood(data,sigma,boundary):
    precision=lambda sigma: np.linalg.inv(sigma)
    likelihood_mu=lambda mu_x,mu_y,sigma: -(data-np.asarray([mu_x,mu_y]).T)\
                                        @ precision(sigma)\
                                        @ (data-np.asarray([mu_x,mu_y]).T).T/2
    dim=50
    mu_x = np.linspace(-boundary, boundary, dim)
    mu_y = np.linspace(-boundary, boundary, dim)
    Z=np.zeros((dim,dim))
    for i,xi in enumerate(mu_x):
        for j,yi in enumerate(mu_y):
            likehood_matrix=likelihood_mu(xi, yi,sigma)
            sum_diagonal=np.einsum('ii', likehood_matrix)
            Z[i,j] = (likehood_matrix.ravel().sum()+sum_diagonal)/2
    X, Y = np.meshgrid(mu_x, mu_y)
    return X, Y, Z

boundary=3
data = np.random.rand(300,2)
X, Y, Z=meanLikelihood(data,sigma,boundary)
plt.figure()
plt.contour(X, Y, Z.T, colors='black');
plt.title('Log-likelihood for the mean $\mu_{x}$ and $\mu_{y}$')
plt.xlabel('$Mean-\mu_{x}$')
plt.ylabel('$Mean-\mu_{y}$')
plt.show()

In [ ]:
def covarianceLikelihood(data,mean,boundary):
    precision=lambda sigma: np.linalg.inv(sigma)
    likelihood_covariance=lambda mean,sigma: -(data-mean)\
                                        @ precision(sigma)\
                                        @ (data-mean).T/2
    dim=50
    sigma_x = np.linspace(-boundary, boundary, dim)
    sigma_y = np.linspace(-boundary, boundary, dim)
    Z=np.zeros((dim,dim))
    for i,xi in enumerate(sigma_x):
        for j,yi in enumerate(sigma_y):
            sigma=np.diag([xi,yi])
            likehood_matrix=likelihood_covariance(mean,sigma)
            sum_diagonal=np.einsum('ii', likehood_matrix)
            Z[i,j] = (likehood_matrix.ravel().sum()+sum_diagonal)/2
    X, Y = np.meshgrid(sigma_x, sigma_y)
    return X, Y, Z

boundary=1
data = np.random.rand(300,2)
mean = np.array([[0,0]])
X, Y, Z=covarianceLikelihood(data,mean,boundary)
plt.figure()
plt.contour(X, Y, Z.T, colors='black');
plt.title('Log-likelihood for the variance $\sigma_{x}$ and $\sigma_{y}$')
plt.xlabel('$\sigma_{x}$')
plt.ylabel('$\sigma_{y}$')
plt.show()

# A2. Explain the key ideas of the two distributions Gamma and Poisson. In the context of conjugate priors, prove the formulas that compute the posterior hyper parameters.



Gamma and the Poisson distribution can form a conjugancy while the former is the prior and the latter describing the likelihood of the data given the target hyper-parameter.

Gamma distribution for hyper-parameter $\lambda$ for a given $\alpha$ and $\beta$ is:

$$p(\lambda|\alpha,\beta)=\frac{\beta^{-\alpha}}{\Gamma(\alpha)}\lambda^{\alpha-1}e^{-\beta\lambda}$$

The likelihood of the of the hyper-parameter $\lambda$ for a given set of independent data $x_{1},x_{2},...,x_{N}$ using a Poisson distribution is:

$$P(x_{1},x_{2},...,x_{N}|\lambda)=\prod_{i=1}^{N}\frac{\lambda^{x_{i}}e^{-\lambda}}{x_{i}!}=\frac{\lambda^{\sum_{i=1}^{N}x_{i}}e^{\sum_{i=1}^{N}(-\lambda)}}{\prod_{i=1}^{N}x_{i}!}=\frac{\lambda^{Nx_{mu}}e^{-N\lambda}}{\prod_{i=1}^{N}x_{i}!}$$

Using Bayes it is possible to get the update on $\lambda$ as:

$$p(\lambda|x_{1},x_{2},...,x_{N})=\frac{p(x_{1},x_{2},...,x_{N}|\lambda)p(\lambda)}{\int p(x_{1},x_{2},...,x_{N},\lambda)d\lambda}=\frac{p(x_{1},x_{2},...,x_{N}|\lambda)p(\lambda)}{p(x_{1},x_{2},...,x_{N})}$$

Since the evidence is not dependent on $\lambda$ it is not important on the maximum a posterior:

$$p(\lambda|x_{1},x_{2},...,x_{N})\propto p(x_{1},x_{2},...,x_{N}|\lambda)p(\lambda)$$

$$p(\lambda|x_{1},x_{2},...,x_{N})\propto \frac{\lambda^{Nx_{mu}}e^{-N\lambda}}{\prod_{i=1}^{N}x_{i}!}p(\lambda)$$


$$p(\lambda|x_{1},x_{2},...,x_{N})\propto \frac{\lambda^{Nx_{mu}}e^{-N\lambda}}{\prod_{i=1}^{N}x_{i}!}\frac{\beta^{-\alpha}}{\Gamma(\alpha)}\lambda^{\alpha-1}e^{-\beta\lambda}$$


$$p(\lambda|x_{1},x_{2},...,x_{N})\propto \frac{\beta^{-\alpha}}{\Gamma(\alpha) \prod_{i=1}^{N}x_{i}!}  \lambda^{Nx_{mu}}e^{-N\lambda}\lambda^{\alpha-1}e^{-\beta\lambda}$$


$$p(\lambda|x_{1},x_{2},...,x_{N})\propto \frac{\beta^{-\alpha}}{\Gamma(\alpha) \prod_{i=1}^{N}x_{i}!}  \lambda^{Nx_{mu}+\alpha-1}e^{-N\lambda-\beta\lambda}$$

Since $\frac{\beta^{-\alpha}}{\Gamma(\alpha) \prod_{i=1}^{N}x_{i}!}$  is not dependent on $\lambda$ can be removed away from the right hand side.

$$p(\lambda|x_{1},x_{2},...,x_{N})\propto   \lambda^{Nx_{mu}+\alpha-1}e^{-\alpha(N+\beta)}$$


$$p(\lambda|x_{1},x_{2},...,x_{N})\propto Gamma(Nx_{mu}+\alpha,N+\beta)$$

Hence the Poisson and the Gamma distribution form a conjugacy. 

# A3. Design an interesting, simple, multimodal distribution with two dimensions. Assume that you can evaluated from this distribution, but you cannot directly sample from it. Implement the MetropolisHastings algorithm, and visualize on top of a contour plot the steps taken by the algorithm. Clearly show when steps are accepted or rejected. 

### Markov Chain Monte-Carlo using Metropolis-Hasting sampling technique

This is a sampling method that utilizes a proposal method to generate samples such as it elliviates the curse of dimensionality by trying to generate samples from high volume of likelihoods.
Instead of drawing samples from the target distribution $\hat{p}(z)$ this method draws samples from a proposal distribution $q(z)$ and computes the ratio $$R(z^*,z^{(\tau)})=\frac{\hat{p}(z^*)q(z^{(\tau)}|z^*)}{\hat{p}(z^{(\tau)})q(z^*|z^{(\tau)})}$$

The acceptation ratio is then upperbounded by one:
$$A(z^*,z^{(\tau)})=min(1,R(z^*,z^{(\tau)}))=min(1,\frac{\hat{p}(z^*)q(z^{(\tau)}|z^*)}{\hat{p}(z^{(\tau)})q(z^*|z^{(\tau)})})$$

Unlike Metropolis algorithm which requires symmetric distribution $q(z^{(\tau)}|z^*)=q(z^*|z^{(\tau)})$ Hasting propose the utilization of non-symmetrical ones.

Eventually the sample $z^{(\tau)}$ is accepted over $z^*$ if $A(z^*,z^{(\tau)})>u$ where $u\sim U[0,1]$ otherwise we reject $z^{(\tau)}$ and keep $z^*$.

Bellow is an example while sampling over the 2D distribution 
$$p(x_1,x_2)=e^{-\frac{(2*x_{1}+sin(2\pi*x_{1}))^2}{2}}\frac{1}{\sqrt{2*\pi*0.1}}e^{-\frac{(x2-x1^3)^2}{0.1}}$$


In [ ]:
# Gibbs sampling applied on a 2D distribution as in the figure bellow 
# where one of the dimensions uses Metropolis-Hastings

pi_1=lambda x1 : np.exp(-((2*x1+np.sin(6.28*x1))**2)/2)
pi_2=lambda x1, x2 : norm.pdf(x2,x1**3,0.1)
pi_1_2=lambda x1, x2 : np.exp(-((2*x1+np.sin(6.28*x1))**2)/2)*norm.pdf(x2,x1**3,0.1)
pi_1_2(1,2)

x = np.linspace(-1, 1, 50)
y = np.linspace(-1, 1, 40)

X1, X2 = np.meshgrid(x, y)
Z = pi_1_2(X1, X2)

plt.contour(X1, X2, Z, colors='black');
plt.xlabel('x1')
plt.ylabel('x2')
plt.show()

In [ ]:
def gibbsMetropolisHasting(sigma=1,
                           nr_iterations=1000):
    x1=[]
    x2=[]
    x1_reject=[]
    x2_reject=[]
    x1.append(0)
    x2.append(0)
    
    
    x1_reject.append(0)
    x2_reject.append(0)
    for i in (range(nr_iterations)):
        
        # Sampling first dimension
        u=np.random.uniform(0,1,1)
        x1_temp=np.random.normal(x1[-1],sigma**2,1)
        ratio=pi_1_2(x1_temp,x2[i])/pi_1_2(x1[i],x2[i])
        a=np.min((1,ratio))
        if u<a:
            x1.append(x1_temp)
        else:
            x1_reject.append(x1_temp)
            x2_reject.append(x2[-1])
            x1.append(x1[-1])
        
        
        # Sampling second dimension
        u=np.random.uniform(0,1,1)
        x2_temp=np.random.normal(x2[-1],sigma**2,1)
        ratio=pi_1_2(x1[-1],x2_temp)/pi_1_2(x1[-1],x2[-1])
        a=np.min((1,ratio))
        if u<a:
            x2.append(x2_temp)
        else:
            x1_reject.append(x1[-1])
            x2_reject.append(x2_temp)
            x2.append(x2[-1])
            
        clear_output(True)
        plt.figure()
        plt.scatter(x1,x2)
        plt.plot(x1,x2)
        plt.contour(X1, X2, Z, colors='black')
        plt.xlabel('x1')
        plt.ylabel('x2')
        plt.title("Gibbs with Metropolis Hasting samples "+str(i)+"/"+str(nr_iterations))
        plt.show()

    return x1,x2,x1_reject,x2_reject

In [ ]:
x1_,x2_,x1_reject,x2_reject=gibbsMetropolisHasting(.5,100)

clear_output(True)
plt.close('all')
plt.figure()
plt.scatter(x1_,x2_)
plt.contour(X1, X2, Z, colors='black')
plt.xlabel('x1')
plt.ylabel('x2')
plt.title("Gibbs with Metropolis Hasting accepted samples")
plt.show()

plt.figure()
plt.scatter(x1_reject,x2_reject)
plt.contour(X1, X2, Z, colors='black')
plt.xlabel('x1')
plt.ylabel('x2')
plt.title("Gibbs with Metropolis Hasting rejected samples")
plt.show()

### Markov Chain Monte-Carlo using Hamiltonian Dynamics
###### Metripolis Hasting algorithm can stuck into one of the modals of the distribution and not expling the rest of the modes.
###### One way to circumvent this my heuristically adjusting the step-size using the spread of the proptoosal distribution. However too large step size leads to a large number of rejections, while a too small step-size makes makes the exploration too slow.
###### in high-dimenbsional space the exploration is nenarly a random-walk behavior thus the exploration is sup-optimal.

To mitigate these drawbacks Hamitonian Monte-Carlo (HMC) utilizes the target distribution and the laws of dynamics in mechanical physics to design adaptive step-size for the proptoosed samples.

The target distribution p(z) is then a modeled using the Gibbs canonical distribution from statistical mechanics as
$$p(z)\propto e^{\frac{-U(z)}{T}} $$ where T is the temperature and U(z) is the energy of the state for the particle at state z.

Apart from the potential energy U(z) this method introduces an additional auxilliary  component kinetic energy K(v) that is dependent on the speed (v) as auxilliary variable.

Eventually the total mechanical energy is:
$$E(z,v)=U(z)+K(v),s.t: K(v)=\sum_{i}\frac{v_{i}^2}{2}$$

The state distribution of the particles is then dependent on the total energy as:

$$p(z,v)\propto e^{\frac{-E(z,v)}{T}}=e^{\frac{-U(z)}{T}}e^{\frac{-K(v)}{T}}\propto p(z)p(v)$$

#### The physicial dynamics of the target distribution through Hamiltonian

In order to sample multiple different positions of the samples inside the energy well defined from E(z,v) we utilize these two physics equations:
$$\frac{\partial z_{i}(t)}{\partial t}=\frac{\partial E(z,v)}{\partial v_{i}}=\frac{\partial K(z)}{\partial v_{i}}$$
$$m\frac{\partial v_{i}(t)}{\partial t}=-\frac{\partial E(z,v)}{\partial z_{i}}=-\frac{\partial U(z)}{\partial z_{i}} $$

Since the energy of the closed system is preserved $E(z,v)=E_{0}$ it is possible to get different samples inside this target distribution while simulating particles whose statistical trajectory is guided by the two equations above.

Sampling the speed (v) is quite simple as it follows a (multivariate) normal distribution:

$$p(v)=e^{\frac{-K(v)}{T}}=e^{\frac{-\sum_{i}mv_{i}}{2T}}=e^{\frac{-mV^{T}V}{2T}}$$

###### In a nutshell

Start the sample moving with a random speed drawn from the normal distribution and stop it.
Continue this proceedure until the sufficient number of samples have been accummulated.

However, numerical solutions to the partial derivative equations (PDE) cannot be solved analytically and their numerical solution does not ensure the preservation of the energy $E(z,v)$.

To mitigate this problem Metropolis Hastings rejections are employed to compensate difference in energy between energy between the start the and the stop of the particle position.

Leapfrog numerical integration offers an numerical integration that is reversible in time.
This reversability ensures the detailed balance.

###### Physical analogy

The trajectory of the particle that roams inside the energy well defined by the target distribution is equivalent to a classical harmonic oscilator without any dampling (conservation of energy).
This is governt by a second had ordinary differential equation (ODE) $z^{''}+x=0$.
To simplify the solution this is converted into two ODEs where $z^{'}=v$.

$$z^{'}=v $$
$$v^{'}=-z $$

These two equations equivalues to a system with kinetic energy $K(v)=\frac{1}{2}v^{2}$ and potential energy $U(z)=\frac{1}{2}z^{2}$.
In the case of multidimensial distribution:
$$K(v)=\frac{v^{T}v}{2}$$
$$U(z)=\frac{z^{T}z}{2}$$

Where the acceptance rate is:

$$A(z^*,z^{(\tau)})=\frac{e^{-U(z^*)-K(z^*)}}{e^{-U(z^{(\tau)})-K(z^{(\tau)})}}=e^{U(z^{(\tau)})-U(z^*)+K(z^{(\tau)})-K(z^*)}$$

where the ODEs $z^{'}=v $, $v^{'}=-x$ can be organised into a matrix formation as $z^{'}=Ax$ where:

$A=\begin{bmatrix}
0 & 1\\
-1 & 0
\end{bmatrix} $

Since the eigenvalues of the matrix $A$ are $i$ and $-i$ the solution of position $x(t)=e^{it}$.
Equation for $x(t)$ is just a circle that does not changes its shape.

###### Euler solution to ODE

Numerical solution to the ODE $z^{'}=Ax$ is from Euler where $\frac{z_{n+1}-z_{n}}{\Delta t}=Az_{n}\to z_{n+1}=z_{n}+\Delta tAz_{n}=(I+\Delta tA)z_{n}=Bz_{n};s.t:B=I+\Delta tA$

###### Leapfrog solution to ODE
Instead of performing the updates simultaneosly, leapfrog method splits this across variables.
It makes one half-step towards the first variable.
Makes a full step towards the second variable using the updated first variable.
Takes one final half step for the first variable using the updated second variable.


###### Concrete example for HMC execution

Consider the target distribution is a multivariate Gaussian with zero mean and covariance $\Sigma$ such as $z\sim N(o,\Sigma)$ and:

$$U(z)=\frac{z^{T}\Sigma^{-1}z}{2}$$

The matrix A in this case is:


$A=\begin{bmatrix}
0 & 1\\
-\Sigma^{-1} & 0
\end{bmatrix} $


In [ ]:
mean=np.array([0,0])
sigma = np.array([[4,1],[1,4]])
precision=lambda cov: np.linalg.inv(sigma)
gaussian=lambda x1,x2: np.exp(-([x1, x2]-mean).T\
                                @ precision(sigma)@([x1, x2]-mean))\
                                /2/np.pi/np.sqrt(np.linalg.det(sigma))

dim=50
boundary=5
x = np.linspace(-boundary, boundary, dim)
y = np.linspace(-boundary, boundary, dim)
Z=np.zeros((dim,dim))
for i,xi in enumerate(x):
    for j,yi in enumerate(y):
        Z[i,j] = gaussian(xi, yi)


X1, X2 = np.meshgrid(x, y)

plt.figure()
plt.contour(X1,X2, Z.T, colors='black');
plt.title('Multivariate Gaussian as target distribution')
plt.xlabel('1-st dim')
plt.ylabel('2-nd dim')
plt.show()

In [ ]:
def leapFrog(sigma,z,v,step, nr_dimensions):
    v=v-step/2*la.inv(sigma)@z
    z_trajectory=[]
    for i in range(nr_dimensions-1):
        z=z+step*v
        v=v-step*la.inv(sigma)@z
        z_trajectory.append(z)
        

    z=z+step*v
    v=v-step/2*la.inv(sigma)@z
    z_trajectory.append(z)
    z_trajectory=np.asarray(z_trajectory)
    
    
    return z_trajectory,z,v

def energyRatio(sigma,z_start,v_start,z_stop,v_stop):
    energy_start=z_start@la.inv(sigma)@z_start+v_start@v_start
    energy_stop=z_stop@la.inv(sigma)@z_stop+v_stop@v_stop
    energy_ratio=energy_stop-energy_start
    return energy_ratio

In [ ]:
nr_iterations = 100
integration_steps = 0.01
nr_integration_steps = 100

samples_rejected = []
samples_accepted = []

# starting position for the particle in the center of the space
z_start = np.array([-0,0])
#samples_accepted[0] = z_start

samples_accepted.append(z_start)
for i in range(nr_iterations+1):
    # Draw a random velocity
    v_start = np.random.normal(0,1,2)

    # Integrate the trajectory of the particle
    z_trajectory,z_stop, v_stop = leapFrog(sigma, z_start, v_start, 
                                 integration_steps, 
                                 nr_integration_steps)
    
    z_start = samples_accepted[i]
    
    # Acceptance ratio
    a = np.exp(-energyRatio(sigma, z_start, v_start, z_stop, v_stop))
    
    # Metropolis-Hasting accept-reject
    r = np.random.rand()
    if r < a:
        samples_accepted.append(z_stop)
    else:
        samples_rejected.append(z_stop)
        samples_accepted.append(z_start)
    
    
    clear_output(True)
    plt.figure()
    plt.contour(X1,X2, Z.T, colors='black');
    plt.plot(z_trajectory[:,0], z_trajectory[:,1],'b')
    plt.plot(z_trajectory[:,0], z_trajectory[:,1],'bx')
    plt.title("Multivariate Gaussian as target distribution\n Iteration "+str(i)+"/"+str(nr_iterations))
    plt.xlabel('1-st dim')
    plt.ylabel('2-nd dim')
    plt.xlim([-boundary, boundary])
    plt.ylim([-boundary, boundary])
    plt.show()
    
    clear_output(True)
    samples_accepted_temp=np.asarray(samples_accepted)
    plt.figure()
    plt.contour(X1,X2, Z.T, colors='black');
    plt.scatter(samples_accepted_temp[1:,0], 
                samples_accepted_temp[1:,1],  
                c='r',
                cmap='Reds')
    plt.title("Multivariate Gaussian as target distribution\n Iteration "+str(i)+"/"+str(nr_iterations))
    plt.xlabel('1-st dim')
    plt.ylabel('2-nd dim')
    plt.xlim([-boundary, boundary])
    plt.ylim([-boundary, boundary])
    plt.show()
        
samples_rejected=np.asarray(samples_rejected)

In [ ]:
samples_accepted=np.asarray(samples_accepted)
clear_output(True)
plt.figure()
plt.contour(X1,X2, Z.T, colors='black');
plt.scatter(samples_accepted[1:,0], 
            samples_accepted[1:,1],  
            c='r', 
            cmap='Reds')
plt.title("Accepted samples from HMC")
plt.xlabel('1-st dim')
plt.ylabel('2-nd dim')
plt.xlim([-boundary, boundary])
plt.ylim([-boundary, boundary])
plt.show()


plt.figure()
plt.contour(X1,X2, Z.T, colors='black');
plt.scatter(samples_rejected[1:,0], 
            samples_rejected[1:,1],  
            c='b')
plt.title("Rejected samples from HMC")
plt.xlabel('1-st dim')
plt.ylabel('2-nd dim')
plt.xlim([-boundary, boundary])
plt.ylim([-boundary, boundary])
plt.show()


## Sequential Monte Carlo (SMC) sampler

Consider a $pdf^2$ in $\mathbb{R}^2$,

\begin{align}
\pi(x)=\frac{1}{Z}\mathbb{1}(-1\leq x_1\leq1)\mathbb{1}(-1\leq x_2\leq1)\sin^2(x_1\pi)\cos^8(x_{2}2\pi)\exp(-5(x_1^2+x_2^2))
\end{align}

where $x = (x_1, x_2)$. 

In [ ]:
import random
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.linalg as la
import matplotlib.pyplot as plt

from tqdm import tqdm
from IPython.display import clear_output
from scipy.stats import norm,uniform,multivariate_normal
np.random.seed(10)
pi_0=lambda x: np.asarray([[1 if xi<=1 and xi>=-1 else 0 for xi in xj]for xj in x])
pi_1=lambda x1 : np.sin(np.pi*x1)**2 
pi_2=lambda x2 : np.cos(2*np.pi*x2)**8
pi_3=lambda x1, x2 : np.exp(-5*(x2**2+x1**2))
pi=lambda x1, x2 : pi_0(x2)*pi_0(x1)*pi_1(x1)*pi_2(x2)*pi_3(x1,x2)
#
pi_00=lambda x:1 if x<=1 and x>=-1 else 0
llk_target=lambda x1,x2:np.log(pi_00(x1)*pi_00(x2)*pi_1(x1)*pi_2(x2)*pi_3(x1,x2)+1e-10)

x = np.linspace(-1, 1, 50)
y = np.linspace(-1, 1, 50)

X1, X2 = np.meshgrid(x, y)
Z = pi(X1, X2)

plt.contour(X1, X2, Z, colors='black');
plt.xlabel('x1')
plt.ylabel('x2')
plt.show()

Implement an SMC sampler to sample from $\pi(x)$ and to estimate the normalizing constant Z. 
Use $N = 100$ particles and an appropriately chosen annealing sequence of K steps, such that it is possible to sample exactly from the initial distribution in the sequence. 

Use ESS-triggered resampling (threshold $0.7N$) and select the annealing sequence in such a way that the average number of SMC iterations between consecu- tive resampling steps is in the order of 10. As a proposal in Metropolis-Hastings, use a Gaussian random walk independent in each dimension with variance $0.02^2$, or similar. Report the results by:

##### Plotting the locations of the particles at initialization, at the final iteration, and at two arbitrarily chosen intermediate iterations (or, if you prefer, make an animation over k).

In [ ]:
mean=np.zeros(2)
mean_1=5*np.ones(2)
cov=(0.2)*np.eye(2)

prior=lambda N: np.random.multivariate_normal(mean, cov, N)
llk_prior=lambda x: np.log(multivariate_normal.pdf(x, mean, cov))
llk_test_target=lambda x: np.log(multivariate_normal.pdf(x, mean_1, cov))

def annealingLikelihood(x,llk_prior,llk_target,k):
    annealed_llk=(1-k)*(llk_prior(x)+1e-10)+k*(llk_target(x[0],x[1])+1e-10)
    return annealed_llk

print(llk_prior(prior(1)))
print(llk_test_target(prior(1)))
annealingLikelihood(prior(1).squeeze(),llk_prior,llk_target,k=1)

In [ ]:
def metropolisHastingGaussianKernel(
    x_conditional,
    prior,
    llk_target,
    llk_proposal,
    N=10,
    nr_smpl=1,
    nr_steps=40):
    """
     N=> nr of samples proposed throught the target distribution
     nr_smpl=> nr of desired samples extracted from the proposed samples using importance weights.
     nr_target_samples=> nr of iterations using in the conditional importance sampling.
    """
    x_star=x_conditional
    x_samples=[]
    
    llk_tg=[]
    llk_pp=[]
    for i in (range(nr_steps)):
        
        # Sample N-1 particles from a proposed distribution
        x=prior(N-1)        
        # Attach to the proposed samples the particle where the kernel is conditioned at
        x=np.vstack((x,x_star))

        # Compute the importance weights based on the proposal and the target distribution
        w=np.asarray([llk_target(x[i])-llk_proposal(x[i]) for i in range(N)]).ravel().squeeze()
        w-=w.max()
        w=np.exp(w)
        w=w/w.sum()
        # Resample the proposed samples based on the importance weights.
        indexes=np.random.multinomial(1,w.squeeze(),nr_smpl).argmax(axis=1)
       
        if np.random.rand()<w[indexes[0]]:
            # Add to the sample array that is approaching the target distribution
            x_star=x[indexes[0]]
            x_samples.append(x_star)
            #print('\Here!')
    return x_star,x_samples

x_samples=metropolisHastingGaussianKernel(
        x_conditional=prior(1),
        prior=prior,
        llk_target=llk_test_target,
        llk_proposal=llk_prior,
        N=10,
        nr_smpl=1,
        nr_steps=20000)[1]
x_samples=np.asarray(x_samples)
plt.scatter(x_samples[:,0],x_samples[:,1])
plt.title('Test Target Distribution')
plt.show()

x_samples=metropolisHastingGaussianKernel(
        prior(1),
        prior,
        llk_target=lambda x: annealingLikelihood(x,llk_prior,llk_target,k=1),
        llk_proposal=lambda x:llk_prior(x),
        N=10,
        nr_smpl=1,
        nr_steps=10000)[1]
x_samples=np.asarray(x_samples)
plt.scatter(x_samples[:,0],x_samples[:,1])
plt.title('Test Target Distribution')
plt.show()

In [ ]:

def rectifyWeights(w):
    w=np.asarray(w)
    if w.squeeze().sum()==0:
        # In the case of the weights being all zero 
        # you restart the values at equal probability
        # such as they all sum up to one.
        w+=np.ones_like(w)/w.ravel().shape[0]
        return w.squeeze(),w.squeeze()
    
    #w+=1e-9
    w_llk=np.log(w)
    
    # Sanity check
    w_llk_normal=w_llk-w_llk.max()-np.log(1e-9)
    w_exp=np.exp(w_llk_normal)
    
    w_norm=w_llk_normal-np.log(w_exp.sum())
    w_llk=np.exp(w_llk_normal)
    
    # Sanity check
    w_norm=w_llk/np.nansum(w_exp)
    w_norm=w_llk/w_llk.sum()
    w_llk,w_norm=w_llk.squeeze(),w_norm.squeeze()
    return w_llk,w_norm

In [ ]:
N=200
nr_iterations=10
X_start=prior(N)
weights_start=np.log(np.ones(N)/N)

k=np.linspace(0,1,nr_iterations)

def ESS(w):
    ess=1/np.sum(w**2)
    return ess

Z_ratio=1
ess_s=[]
for j in tqdm(range(nr_iterations)):
    weights_start=[weights_start[i]\
                   +annealingLikelihood(X_start[i],llk_prior,llk_target,k=k[j])\
                   -annealingLikelihood(X_start[i],llk_prior,llk_target,k=k[j-1])
                   for i in range(N)]
    weights_start=np.asarray(weights_start).ravel().squeeze()
    #weights_start-=weights_start.ravel().max()
    weights=np.exp(weights_start)
    weights=rectifyWeights(weights)[-1].squeeze()
    ess=ESS(weights)
    ess_s.append(ess)

    # if effective sample is low start resampling to remove, 
    # the particles with low weights.
    if ess<0.7*N:
        # Resample the particles based on the adapted distribution
        indexes=np.random.multinomial(1,weights,N).argmax(axis=1)
        X_start=X_start[indexes]
        weights_start=np.log(np.ones(N)/N)
    
    X_new=[metropolisHastingGaussianKernel(   
                    X_start[i],
                    prior,
                    llk_target=lambda x:annealingLikelihood(x,llk_prior,llk_target,k=k[j]),
                    llk_proposal=lambda x: llk_prior(x),
                    N=10,
                    nr_smpl=1,
                    nr_steps=100)[0]  for i in (range(N))]
    X_start=np.asarray(X_new)
    Z_ratio*=weights.ravel().sum()
    clear_output(True)
    plt.figure(1)
    plt.scatter(X_start[:,0],X_start[:,1])
    plt.contour(X1, X2, Z, colors='black')
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.title("SMC sampler")
    plt.show()